In [2]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime, date

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/muef/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
    dtm_path = "/home/galm/software/dtm/dtm/main"
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')
    dtm_path = "/home/finn/dtm/dtm/main"

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import scoping.models as sm
from parliament.tasks import do_search, run_tm, combine_searches
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import * 
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.db.models.functions import TruncDate, TruncMonth, TruncYear

In [14]:
run_id = 2350
no_topwords = 20

dts = DocTopic.objects.filter(run_id=run_id)
topic_period_scores = dts.values('topic__title','ut__document__parlperiod__n').annotate(score = Sum('score'))
df = pd.DataFrame.from_dict(topic_period_scores)
df.columns = ['score', 'topic', 'parlperiod']
df_scores = df.pivot(index='parlperiod', columns='topic', values='score')
df_scores.loc['peak_pos'] = [df_scores[topic].idxmax() for topic in df_scores.columns]
df_scores

topic,European Coal and Steel Community,agricultural policy,budget,budget 2,climate protection,coal mining,coal policy & prices,debate government policy,economic policy,economic policy & participation,...,procedural,procedural 2,research & development,reunification,risks of nuclear energy,stop words,structural adjustment,subsidy reduction,tax policy,transport policy
parlperiod,,,,,,,,,,,,,,,,,,,,,
1,82349.501906,6075.067415,8808.920749,568.410879,133.509431,1203.456920,32549.026092,150.058039,19527.988686,19789.142870,...,29570.854228,5753.340716,610.013087,922.281888,212.766429,34871.676295,3188.165862,358.034158,10550.394510,8859.196201
2,22120.041003,11746.123703,10371.860697,892.688278,17.090793,1861.159903,17294.711048,169.451466,24322.678377,12796.307571,...,32409.030139,6726.732726,835.735827,313.188712,3719.846395,27966.162279,826.408031,139.018589,4918.252851,25613.350759
3,4961.588324,6062.651433,15792.286786,1085.678927,41.316981,4956.503596,22657.844169,117.760484,17077.986502,10858.057584,...,17488.523667,5987.360267,536.776650,186.005579,1175.932566,22525.190183,630.533994,280.763891,4515.475153,12501.701518
4,5673.928909,7495.179284,13990.198525,773.740105,11.591260,11738.744518,8945.621078,49.152969,12161.850947,5196.464497,...,18788.790099,4324.711398,413.059816,171.682336,325.825542,15703.164858,838.794467,116.232722,2632.750128,7260.055793
5,5829.877829,9735.347258,23592.621252,2030.585239,28.205642,23042.731360,5556.741521,165.071746,27829.584234,4208.738350,...,41199.330774,12518.176387,3264.254635,237.413062,2606.525427,28573.695703,4145.890465,416.471753,4268.223532,7281.799161
6,926.608580,1065.395238,5049.703827,1616.695931,14.282931,1748.231997,1359.335713,50.174365,3389.087215,1958.799231,...,7000.517929,8427.062292,1072.070947,89.480913,555.089428,4633.578567,1129.971308,28.598703,1179.077130,1151.901138
7,1866.178929,602.035243,9564.657965,7020.779149,40.507234,3401.591728,4003.848757,101.867724,11545.858097,3993.426266,...,25420.252463,16989.894766,6434.143469,431.435108,5428.793496,14538.858250,1081.705529,98.030403,3058.782732,1407.219583
8,1697.505582,839.932809,8253.339384,11074.263889,78.545343,2936.949468,2654.949400,861.655985,16766.369416,2914.362513,...,34080.125027,7794.837897,6655.047279,435.831256,10410.543462,22886.779974,3354.051169,534.486899,5575.587174,1195.021452
9,793.854367,716.739409,6336.952230,13496.975072,45.622223,945.164826,947.573386,735.565518,10582.879805,2620.131920,...,16739.629201,1711.687473,3759.572864,748.754320,5129.947894,18802.583964,1881.250505,495.473508,2614.180030,556.799025


In [12]:
def get_topwords_weighted_by_periodscore(topic_id, no_topwords):

    topic = Topic.objects.get(id=topic_id)
    terms = TopicTerm.objects.filter(topic=topic).values('term__title', 'PY', 'score').order_by('term', 'PY')
    terms_df = pd.DataFrame.from_dict(terms).rename(columns={"term__title": "term", "PY": "period", "score": 'term score'})

    period_scores = TopicTimePeriodScores.objects.filter(
                            topic=topic
                        ).order_by('period__n').values('score', 'period__n')
                    # period links to TimePeriod
    period_scores_df = pd.DataFrame.from_dict(period_scores).rename(columns={"score": "period score", "period__n": "period"})
    terms_joined = terms_df.join(period_scores_df, 'period', lsuffix='_left', rsuffix='_right')
    terms_joined['score product'] = terms_joined['term score'] * terms_joined['period score']
    if terms_joined.isnull().values.any():
        print("join did not work")
    topword_df = terms_joined[['term', 'score product']].groupby(['term']).sum().sort_values('score product', ascending=False)
    
    # note: table join in Database not working because PY and run_id are not ForeignKeys but db_index=True
    # probably this has been chosen for speeding up bulk insertion
    
    return list(topword_df.index[:no_topwords])

In [15]:
# topic labeling: manual and automatic

labels = {}

df = pd.DataFrame(columns=['label', 'top words weighted', 'top words avg', 'top words peak', 'score pct', 'score'])

get_input = False
# set to 0 if current topic labels should be kept
no_label_topwords = 0
assign_topwords = True

for topic in Topic.objects.filter(run_id = run_id).order_by('-score'):
    stat = RunStats.objects.get(run_id=run_id)

    print("Topic #{}".format(topic.id))
    
    if stat.method == 'BD':
        
        terms = Term.objects.filter(topicterm__topic=topic).values('title').annotate(
                    summed_score = Sum(F('topicterm__score'))).order_by('-summed_score')[:no_topwords]
        term_list = [t['title'] for t in terms]
        print(", ".join(term_list))
        
        # weight by document scores per period
        weighted_term_list = get_topwords_weighted_by_periodscore(topic.id, 20)

        print("weighted by topic scores:")
        print(", ".join(weighted_term_list))
        
        # top words from peak
        py = df_scores.loc['peak_pos', topic.title] - 1
        # note: PY starts with 0
        
        peak_terms = TopicTerm.objects.filter(topic=topic, PY=py
                                            ).order_by('-score')[:no_topwords].select_related('term')
        
        peak_term_list = [t.term.title for t in peak_terms]
        print("peak terms:")
        print(", ".join(peak_term_list))
        
        if assign_topwords:
            print("saving weighted topword list")
            topic.top_words = weighted_term_list
            topic.save()

    else:
        print("Top words: {}".format(topic.top_words))

    if get_input:
        label = input("topic label: ")
        if label:
            labels[topic.id] = label
            topic.title = label
            topic.save()
              
    elif no_label_topwords:
        topic.title = "{" + ", ".join(weighted_term_list[:no_label_topwords]) + "}"
        topic.save()
        print(topic.title)
    
    else:
        print("topic label: {}".format(topic.title))
    
    print("")
    
    df.loc[topic.id] = {"top words weighted": ', '.join(weighted_term_list),
                        "top words avg": ", ".join(term_list),
                        "top words peak": ", ".join(peak_term_list),
                        "score": topic.score, "label": topic.title, "score pct": 0}

print(labels)
df['score pct'] = 100 * df['score'] / df['score'].sum()
df = df.sort_values(by=['score'], ascending=False)

Topic #184089
tun, wiss, wirklich, red, gar, darub, natur, richtig, eigent, namlich, vielleicht, weiss, uberhaupt, seh, stell, ding, sei, einfach, sagt, gemacht
weighted by topic scores:
tun, wirklich, wiss, red, gar, darub, natur, richtig, eigent, namlich, vielleicht, weiss, ding, uberhaupt, seh, sei, sagt, einfach, stell, gemacht
peak terms:
ding, vielleicht, wirklich, gar, wiss, seh, weiss, sei, darub, tun, richtig, sagt, namlich, wort, wenig, genau, eben, kommt, uberhaupt, eigent
saving weighted topword list
topic label: stop words

Topic #184090
darauf, darub, haus, bereich, punkt, debatt, bundesregier, entscheid, deutlich, diskussion, darf, richtig, natur, zusammenhang, wichtig, ausdruck, stell, seit, bundestag, lass
weighted by topic scores:
darauf, haus, darub, darf, bereich, punkt, bundesregier, entscheid, debatt, zusammenhang, diskussion, ausdruck, deutlich, seit, richtig, lass, stell, natur, meinung, klar
peak terms:
haus, darauf, darf, darub, entscheid, lass, bundesregier, 

wettbewerb, strom, kost, kraftwerk, preis, strompreis, energi, kilowattstund, prozent, erneuerbar, deutschland, netz, verbrauch, pfennig, anlag, energiew, markt, energieversorg, beispiel, hoh
weighted by topic scores:
strom, energi, erneuerbar, wettbewerb, prozent, energiew, kost, netz, kraftwerk, deutschland, strompreis, ausbau, verbrauch, preis, eeg, hoh, energieversorg, anlag, letzt, kilowattstund
peak terms:
energi, erneuerbar, strom, energiew, prozent, netz, ausbau, deutschland, kost, kraftwerk, verbrauch, eeg, energieversorg, hoh, strompreis, letzt, preis, wettbewerb, versorgungssich, bereich
saving weighted topword list
topic label: energy transition & power market

Topic #184113
million, tonn, staatssekretar, kohl, pro, deutsch, preis, bundeswirtschaftsminist, etwa, bundesregier, hoh, zahl, heizol, verbrauch, seit, wirtschaft, industri, verfug, bekannt, lag
weighted by topic scores:
million, tonn, kohl, bundeswirtschaftsminist, deutsch, preis, bundesregier, hoh, etwa, staatssek

weighted by topic scores:
kernenergi, kernkraftwerk, technik, anlag, technisch, bundesregier, nutzung, brut, schnell, betrieb, reaktor, tschernobyl, deutsch, bundesrepubl, entwickl, technologi, gefahr, wissenschaft, radioaktiv, hochtemperaturreaktor
peak terms:
kernenergi, kernkraftwerk, technisch, bundesregier, anlag, technik, brut, nutzung, schnell, entwickl, betrieb, hochtemperaturreaktor, bundesrepubl, deutsch, wissenschaft, bau, radioaktiv, gefahr, hoh, friedlich
saving weighted topword list
topic label: risks of nuclear energy

Topic #184088
kernenergi, energiepolit, grun, deutschland, ministerprasident, ausstieg, kernkraftwerk, spd, bundesregier, kohl, entscheid, kons, niedersachs, bau, landesregier, nordrheinwestfal, klar, deutsch, energi, partei
weighted by topic scores:
kernenergi, energiepolit, deutschland, grun, kernkraftwerk, spd, ausstieg, kohl, entscheid, ministerprasident, bundesregier, kons, niedersachs, landesregier, nordrheinwestfal, bau, klar, arbeitsplatz, deutsch,

In [16]:
df

,label,top words weighted,top words avg,top words peak,score pct,score
184089,stop words,"tun, wirklich, wiss, red, gar, darub, natur, r...","tun, wiss, wirklich, red, gar, darub, natur, r...","ding, vielleicht, wirklich, gar, wiss, seh, we...",10.672064,419821.116446
184090,procedural,"darauf, haus, darub, darf, bereich, punkt, bun...","darauf, darub, haus, bereich, punkt, debatt, b...","haus, darauf, darf, darub, entscheid, lass, bu...",9.348242,367744.157874
184081,economic policy,"wirtschaft, unternehm, entwickl, deutsch, mark...","unternehm, wirtschaft, deutsch, deutschland, m...","wirtschaft, entwickl, bereich, wirtschaftspoli...",5.492435,216063.202495
184105,federal government,"bundeskanzl, partei, regier, deutsch, sozialde...","bundeskanzl, deutsch, partei, deutschland, reg...","regier, bundeskanzl, volk, partei, sozialdemok...",4.037749,158838.262969
184102,policy statements,"deutsch, volk, bundesregier, freiheit, wirtsch...","deutsch, volk, bundesregier, freiheit, deutsch...","deutsch, deutschland, gemeinsam, wirtschaft, f...",3.989937,156957.446933
184106,budget,"milliard, million, bund, haushalt, hoh, rund, ...","milliard, bund, million, hoh, rund, haushalt, ...","million, milliard, bund, haushalt, hoh, bundes...",3.783282,148827.974002
184086,European Coal and Steel Community,"europa, deutsch, vertrag, franzos, frankreich,...","europa, union, gemeinschaft, deutsch, vertrag,...","deutsch, schumanplan, deutschland, vertrag, fr...",3.685528,144982.504003
184098,international cooperation,"europa, gemeinsam, international, deutschland,...","europa, gemeinsam, international, staat, gemei...","deutschland, wichtig, international, gemeinsam...",3.627137,142685.511866
184108,energy policy,"energi, erneuerbar, energiepolit, kohl, bundes...","energi, erneuerbar, kohl, energiepolit, energi...","energiepolit, bundesregier, energi, kohl, kern...",3.422472,134634.299691
184097,procedural 2,"gesetz, wirtschaft, bundesregier, entwurf, bun...","gesetz, wirtschaft, bundesregier, entwurf, bun...","ewg, rechtzeit, gesetz, endgult, wirtschaft, b...",3.160606,124332.949316


In [9]:
# export list to spread sheet
fname = "./plots/topic_analysis_{}/0_topic_list+peak_topwords_run{}.xlsx".format(run_id, run_id)
writer = pd.ExcelWriter(fname, engine='xlsxwriter')

df.to_excel(writer)

Exception ignored in: <bound method Workbook.__del__ of <xlsxwriter.workbook.Workbook object at 0x7f40722a53c8>>
Traceback (most recent call last):
  File "/home/galm/software/django/tmv/venv/lib/python3.6/site-packages/xlsxwriter/workbook.py", line 154, in __del__
    raise Exception("Exception caught in workbook destructor. "
Exception: Exception caught in workbook destructor. Explicit close() may be required for workbook.


In [8]:
# import and set topic labels from spread sheet

topic_dir = "plots/topic_analysis_{}/".format(run_id)
topic_classification = pd.read_csv(topic_dir + '0_topic_list_run{}_labeled.csv'.format(run_id), index_col=0)
topic_classification

,label,label short,classification,label Finn,Discussion with Jan,Jan,Jan.1,Jan period,top words weighted,top words avg,top words peak,score pct,score
184089,stop words,stop words,procedural and general politics,stopwords,NaN,stop words,NaN,NaN,"tun, wirklich, wiss, red, gar, darub, natur, r...","tun, wiss, wirklich, red, gar, darub, natur, r...","ding, vielleicht, wirklich, gar, wiss, seh, we...",10.672064,419821.116446
184090,procedural,procedural,procedural and general politics,procedural,NaN,procedural,NaN,NaN,"darauf, haus, darub, darf, bereich, punkt, bun...","darauf, darub, haus, bereich, punkt, debatt, b...","haus, darauf, darf, darub, entscheid, lass, bu...",9.348242,367744.157874
184081,economic policy,economic policy,economy and finance,economic policy,NaN,Wirtschaftspolitik/ -entwicklung,NaN,NaN,"wirtschaft, unternehm, entwickl, deutsch, mark...","unternehm, wirtschaft, deutsch, deutschland, m...","wirtschaft, entwickl, bereich, wirtschaftspoli...",5.492435,216063.202495
184105,federal government,federal government,procedural and general politics,procedural 2,NaN,Bundesregierung,Procedere,NaN,"bundeskanzl, partei, regier, deutsch, sozialde...","bundeskanzl, deutsch, partei, deutschland, reg...","regier, bundeskanzl, volk, partei, sozialdemok...",4.037749,158838.262969
184102,policy statements,policy statements,procedural and general politics,economic system ?,NaN,NaN,Grundsatzreden,NaN,"deutsch, volk, bundesregier, freiheit, wirtsch...","deutsch, volk, bundesregier, freiheit, deutsch...","deutsch, deutschland, gemeinsam, wirtschaft, f...",3.989937,156957.446933
184106,budget,budget,economy and budget,budget,NaN,Haushaltspolitik/ Bundeshaushalt,NaN,1 bis 5/6 - hält sich aber,"milliard, million, bund, haushalt, hoh, rund, ...","milliard, bund, million, hoh, rund, haushalt, ...","million, milliard, bund, haushalt, hoh, bundes...",3.783282,148827.974002
184086,European Coal and Steel Community,ECSC,international and regional,German-French economic policy,NaN,Europäische (Wirtschafts-)gemeinschaft,Deutsch-französische Wirtschaftszusammenarbeit,NaN,"europa, deutsch, vertrag, franzos, frankreich,...","europa, union, gemeinschaft, deutsch, vertrag,...","deutsch, schumanplan, deutschland, vertrag, fr...",3.685528,144982.504003
184098,international cooperation,international cooperation,international and regional,Europe,international,Internationale Politik,Europäische Politk,NaN,"europa, gemeinsam, international, deutschland,...","europa, gemeinsam, international, staat, gemei...","deutschland, wichtig, international, gemeinsam...",3.627137,142685.511866
184108,energy policy,energy policy,energy,energy policy,NaN,Energiepolitik,look at topic score development and explain,NaN,"energi, erneuerbar, energiepolit, kohl, bundes...","energi, erneuerbar, kohl, energiepolit, energi...","energiepolit, bundesregier, energi, kohl, kern...",3.422472,134634.299691
184097,procedural 2,mixed,procedural and general politics,Mixed ?,Gesetzgebung?,Gesetzgebung,EWG highlighted,NaN,"gesetz, wirtschaft, bundesregier, entwurf, bun...","gesetz, wirtschaft, bundesregier, entwurf, bun...","ewg, rechtzeit, gesetz, endgult, wirtschaft, b...",3.160606,124332.949316


In [10]:
for topic in Topic.objects.filter(run_id = run_id).order_by('-score'):
    topic.title = topic_classification.loc[topic.id, "label"]
    topic.save()

In [16]:
# export as latex table
pd.set_option('max_colwidth', 500)

fname = "./plots/topic_analysis_{}/0_topic_list.tex".format(run_id)

with open(fname,'w') as tf:
    tf.write(df.to_latex(index=False, columns=["label", "top words", "score pct"],
                         column_format='L{3cm}L{10.5cm}r', float_format="{:0.2f}".format,
                        longtable=True).replace('\n',
                                                '\n\\caption{List of topics with labels, top words and scores (in percent of total score).}
                                                '\n\label{tab:topic-list}\n\small\\\\\n', 1))

SyntaxError: EOL while scanning string literal (<ipython-input-16-f0b5b4e3004e>, line 9)

In [17]:
# modify single labels:

topic = Topic.objects.get(id=178480)
topic.title = 'nuclear phase out 1'
topic.save()